<a href="https://colab.research.google.com/github/Norod/hebrew-gpt_neo/blob/main/hebrew-gpt_neo-tiny/Norod78_hebrew_gpt_neo_tiny_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers==0.10.2 transformers==4.5.1

     |████████████████████████████████| 3.3MB 8.6MB/s 
     |████████████████████████████████| 2.1MB 34.5MB/s 
     |████████████████████████████████| 901kB 67.4MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained("Norod78/hebrew-gpt_neo-tiny")
model = AutoModelForCausalLM.from_pretrained("Norod78/hebrew-gpt_neo-tiny", pad_token_id=tokenizer.eos_token_id)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [3]:
prompt_text = "אני אוהב שוקולד ועוגות"
max_len = 512
sample_output_num = 3
seed = 1000

In [4]:
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

print(f"device: {device}, n_gpu: {n_gpu}")

np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

model.to(device)

encoded_prompt = tokenizer.encode(
    prompt_text, add_special_tokens=False, return_tensors="pt")

encoded_prompt = encoded_prompt.to(device)

if encoded_prompt.size()[-1] == 0:
        input_ids = None
else:
        input_ids = encoded_prompt

print("input_ids = " + str(input_ids))

if input_ids != None:
  max_len += len(encoded_prompt[0])
  if max_len > 2048:
    max_len = 2048

print("Updated max_len = " + str(max_len))

device: cuda, n_gpu: 1
input_ids = tensor([[ 2202, 22283, 10984,   631,  5679]], device='cuda:0')
Updated max_len = 517


In [5]:
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=max_len, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=sample_output_num
)

print(100 * '-' + "\nOutput:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("\n{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
  print("\n" + 100 * '-')

----------------------------------------------------------------------------------------------------
Output:
----------------------------------------------------------------------------------------------------

0: אני אוהב שוקולד ועוגות שוקולד. אני רוצה שכולם יקנו לי את זה".
"לצערי, יש אנשים שמבינים כמה חשוב זה להכין שוקולד. הם רוצים לראות משהו אחר, וזה לא קשור להוליווד ולא לקולנוע, זו לא רק השפה שלי. כשאני רואה סרט מסוים, אני רואה את זה".


לשם כך צריך אישורי רופא, אישור מרופא, אישור של תרופות מרשם, אישור של רופא משפחה, אישור של רופא משפחה.
לא כל רופא מקבל אישור מרופא שיניים (רופא שיניים/שיניים או אישורי רופא שיניים) של רופא שיניים. חשוב לזכור, לא כל רופא מקבל אישור מרופא שיניים.
רוב האנשים מקבלים טופס"דוח רופא שיניים"(רופא שיניים /רופא שיניים), אישור של רופא שיניים. חשוב לדעת, לא כל רופא מקבל אישור מרופא שיניים.
אישור רופא שיניים: גם מי שעבר הכשרה במקצוע הרופא הוא אישורי רופא שיניים. כמו כן, יש לבדוק את כל מי שעבר הכשרה במקצוע הרופא – זה כמו תעודת מרופא שיניים.
כל מי שעבר הכשרה בענף 